In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
seoul=pd.read_csv('/content/drive/MyDrive/seoul.csv')
df_경복궁=pd.read_csv('/content/drive/MyDrive/경복궁.csv')
df_국립중앙박물관=pd.read_csv('/content/drive/MyDrive/국립중앙박물관.csv')
df_남산골한옥마을=pd.read_csv('/content/drive/MyDrive/남산골한옥마을.csv')
df_덕수궁=pd.read_csv('/content/drive/MyDrive/덕수궁.csv')
df_서대문자연사박물관=pd.read_csv('/content/drive/MyDrive/서대문자연사박물관.csv')
df_서대문형무소역사관=pd.read_csv('/content/drive/MyDrive/서대문형무소역사관.csv')
df_서울시립미술관=pd.read_csv('/content/drive/MyDrive/서울시립미술관.csv')
df_선릉정릉=pd.read_csv('/content/drive/MyDrive/선릉정릉.csv')
df_종묘=pd.read_csv('/content/drive/MyDrive/종묘.csv')
df_창경궁=pd.read_csv('/content/drive/MyDrive/창경궁.csv')
df_창덕궁=pd.read_csv('/content/drive/MyDrive/창덕궁.csv')
df_태릉강릉조선왕릉전시관=pd.read_csv('/content/drive/MyDrive/태릉강릉조선왕릉전시관.csv')
df_트릭아이미술관=pd.read_csv('/content/drive/MyDrive/트릭아이미술관.csv')
df_헌릉인릉=pd.read_csv('/content/drive/MyDrive/헌릉인릉.csv')

In [3]:
import warnings
warnings.filterwarnings('ignore')

Case1) 계절별로 나누어서 분석을 할 경우
--

In [4]:
class grand_parents_option:
    def discomfort_index(df_1): #불쾌지수 옵션
        df_1=df_1.reset_index()
        num=len(df_1.index)
        df_1['불쾌지수']=0
        for i in range(0,num):
            T =df_1['평균 기온(°C)'][i]
            H =df_1['평균 상대습도(%)'][i]
            discomfort_index = (9/5 * T) - (0.55 * (1 - H/100) * ((9/5 * T) - 26)) + 32
            df_1['불쾌지수'][i]=discomfort_index
  
        return df_1

    def summer_feature(df_2): #여름철 체감온도 옵션
        df_2=df_2.reset_index()
        num=len(df_2.index)
        df_2['체감온도']=0
        for i in range(0,num):
            H=df_2['평균 상대습도(%)'][i]
            T=df_2['평균 기온(°C)'][i]
            SMSensibleTemperature = -42.379 + (2.04901523*T) + (10.14333127 * H) - (0.22475541 * T * H) - (6.83783 * (10 ** -3) * (T ** 2)) - (5.481717 * (10 ** -2)*(H**2)) + (1.22874 * (10 ** -3) * (T ** 2) * H) + (8.5282 * (10 ** -4) * T * (H **2)) - (1.99 * (10**-6) * (T**2) * (H**2))
            df_2['체감온도'][i]=SMSensibleTemperature

        return df_2

    def winter_feature(df_3): #겨울철 체감온도 옵션
        df_3=df_3.reset_index()
        num=len(df_3.index)
        df_3['체감온도']=0
        for i in range(0,num):
            T = df_3['평균 기온(°C)'][i] 
            V = df_3['평균 풍속(m/s)'][i]
            WTSensibleTemperature = 13.12 + (0.6215 * T) - (11.37 * 0.16 ** V) + (0.3965 *V *0.16**T)
            df_3['체감온도'][i]=WTSensibleTemperature

        return df_3

In [5]:
class grand_parents:
    def season_columns(df):
        df=df
        df['season_columns']=0
        for i in range(0,108):
            month=df['month'][i]
            if 3<=month<=5:
                df['season_columns'][i]=1
            elif 6<=month<=8:
                df['season_columns'][i]=2
            elif 9<=month<=11:
                df['season_columns'][i]=3
            else:
                df['season_columns'][i]=4
        return df
        
    def make_season(df):
        df=grand_parents.season_columns(df)
        spring=df.loc[df['season_columns']==1]
        summer=df.loc[df['season_columns']==2]
        fall=df.loc[df['season_columns']==3]
        winter=df.loc[df['season_columns']==4]

        return spring, summer, fall, winter

    def make_dataframe(df):
        spring_1=grand_parents.make_season(df)[0]
        summer_1=grand_parents.make_season(df)[1]
        fall_1=grand_parents.make_season(df)[2]
        winter_1=grand_parents.make_season(df)[3]

        spring=grand_parents_option.discomfort_index(spring_1)
        summer_2=grand_parents_option.discomfort_index(summer_1)
        fall=grand_parents_option.discomfort_index(fall_1)
        winter_2=grand_parents_option.discomfort_index(winter_1)

        summer=grand_parents_option.summer_feature(summer_2)
        winter=grand_parents_option.winter_feature(winter_2)

        return spring, summer, fall, winter

Case2) 계절별로 나누지 않고 적용할 경우
--

In [ ]:
!pip install wordcloud

In [ ]:
!pip install konlpy wordcloud

In [8]:
import xgboost
import wordcloud
from wordcloud import WordCloud, ImageColorGenerator
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
class parents:
    def modeling(x_train, x_test, y_train):
        xgb = xgboost.XGBRegressor(random_state=42)
        xgb.fit(x_train, y_train)
        y_pred=xgb.predict(x_test)
        train_score=xgb.score(x_train, y_train)
        return train_score, y_pred

    def 경복궁(today):
        df=df_경복궁
        x_train=df.drop(['경복궁'], axis=1)
        x_test=today[['date', 'month', '평균 기온(°C)', '최다 풍향(16방위)', '일강수량(mm)',
       '합계 일조 시간(hr)', '9-9강수(mm)', '평균 전운량(10분위)', '합계 소형증발량(mm)',
       '최저 해면기압 시각(hhmi)', '최저 기온 시각(hhmi)', '열지수', '평균 10cm 지중온도(°C)',
       '평균 상대습도(%)', '최대 순간풍속 시각(hhmi)', '최소 상대습도(%)', '풍정합(100m)',
       '최고 기온 시각(hhmi)', '평균 20cm 지중온도(°C)', '합계 일사량(MJ/m2)']]
        y_train=df['경복궁']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 국립중앙박물관(today):
        df=df_국립중앙박물관
        x_train=df.drop(['국립중앙박물관'], axis=1)
        x_test=today[['date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)', '평균 중기압(hPa)',
       '최다 풍향(16방위)', '최저 해면기압(hPa)', '최대 순간풍속 풍향(16방위)']]
        y_train=df['국립중앙박물관']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 남산골한옥마을(today):
        df=df_남산골한옥마을
        x_train=df.drop(['남산골한옥마을'], axis=1)
        x_test=today[['date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)', '합계 소형증발량(mm)',
       '최저 초상온도(°C)', '최다 풍향(16방위)', '9-9강수(mm)']]
        y_train=df['남산골한옥마을']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 덕수궁(today):
        df=df_덕수궁
        x_train=df.drop(['덕수궁'], axis=1)
        x_test=today[['date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)', '0.5m 지중온도(°C)',
       '최대 풍속(m/s)', '합계 일사량(MJ/m2)']]
        y_train=df['덕수궁']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 서대문자연사박물관(today):
        df=df_서대문자연사박물관
        x_train=df.drop(['서대문자연사박물관'], axis=1)
        x_test=today[['date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)', '합계 소형증발량(mm)',
       '5.0cm 지중온도(°C)', '가조시간(hr)', '최대 순간풍속(m/s)', '평균 지면온도(°C)',
       '최대 풍속(m/s)']]
        y_train=df['서대문자연사박물관']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 서대문형무소역사관(today):
        df=df_서대문형무소역사관
        x_train=df.drop(['서대문형무소역사관'], axis=1)
        x_test=today[['date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)',
       '최저 해면기압 시각(hhmi)', '평균 전운량(10분위)']]
        y_train=df['서대문형무소역사관']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 서울시립미술관(today):
        df=df_서울시립미술관
        x_train=df.drop(['서울시립미술관 본관'], axis=1)
        x_test=today[['date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)', '평균 해면기압(hPa)',
       '5.0cm 지중온도(°C)', '최저 초상온도(°C)', '1시간 최다 일사량(MJ/m2)', '9-9강수(mm)',
       '최대 풍속(m/s)', '최저 기온(°C)', '평균 30cm 지중온도(°C)', '평균 5cm 지중온도(°C)']]
        y_train=df['서울시립미술관 본관']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 선릉정릉(today):
        df=df_선릉정릉
        x_train=df.drop(['선릉·정릉'], axis=1)
        x_test=today[['date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)',
       '평균 20cm 지중온도(°C)', '0.5m 지중온도(°C)', '최대 풍속 풍향(16방위)']]
        y_train=df['선릉·정릉']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 종묘(today):
        df=df_종묘
        x_train=df.drop(['종묘'], axis=1)
        x_test=today[['date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)', '0.5m 지중온도(°C)',
       '최저 기온(°C)']]
        y_train=df['종묘']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 창경궁(today):
        df=df_창경궁
        x_train=df.drop(['창경궁'], axis=1)
        x_test=today[['date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)',
       '최대 순간풍속 시각(hhmi)', '최저 해면기압 시각(hhmi)', '최대 순간풍속 풍향(16방위)', '풍정합(100m)']]
        y_train=df['창경궁']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 창덕궁(today):
        df=df_창덕궁
        x_train=df.drop(['창덕궁'], axis=1)
        x_test=today[['date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)',
       '최대 순간풍속 풍향(16방위)', '평균 상대습도 시각(hhmi)', '최대 풍속(m/s)', '합계 일사량(MJ/m2)']]
        y_train=df['창덕궁']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 태릉강릉조선왕릉전시관(today):
        df=df_태릉강릉조선왕릉전시관
        x_train=df.drop(['태릉 ·  강릉 · 조선왕릉전시관'], axis=1)
        x_test=today[['date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)', '최저 기온(°C)',
       '평균 중기압(hPa)', '가조시간(hr)', '평균 이슬점온도(°C)']]
        y_train=df['태릉 ·  강릉 · 조선왕릉전시관']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 트릭아이미술관(today):
        df=df_트릭아이미술관
        x_train=df.drop(['트릭아이미술관'], axis=1)
        x_test=today[[ 'date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)',
       '평균 20cm 지중온도(°C)', '1.5cm 지중온도(°C)', '9-9강수(mm)']]
        y_train=df['트릭아이미술관']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

    def 헌릉인릉(today):
        df=df_헌릉인릉
        x_train=df.drop(['헌릉ㆍ인릉'], axis=1)
        x_test=today[[ 'date', 'month', '최고 기온(°C)', '최소 상대습도(%)', '평균 기온(°C)',
       '평균 상대습도(%)', '일강수량(mm)', '평균 풍속(m/s)', '합계 일조 시간(hr)',
       '평균 중하층운량(10분위)', '최대 풍속(m/s)', '최고 해면 기압(hPa)', '평균 30cm 지중온도(°C)',
       '최저 해면기압(hPa)', '평균 5cm 지중온도(°C)']]
        y_train=df['헌릉ㆍ인릉']
        train_score=parents.modeling(x_train, x_test, y_train)[0]
        y_pred=parents.modeling(x_train, x_test, y_train)[1]
        return train_score, y_pred

In [ ]:
class child:
    def make_dataframe(today):
        df=pd.DataFrame(columns=['경복궁','국립중앙박물관','남산골한옥마을',
                                    '덕수궁','서대문자연사박물관','서대문형무소역사관',
                                    '서울시립미술관','선릉정릉','종묘','창경궁',
                                    '창덕궁','태릉강릉조선왕릉전시관','트릭아이미술관',
                                    '헌릉인릉'])
        경복궁=parents.경복궁(today)[1]
        국립중앙박물관=parents.국립중앙박물관(today)[1]
        남산골한옥마을=parents.남산골한옥마을(today)[1]
        덕수궁=parents.덕수궁(today)[1]
        서대문자연사박물관=parents.서대문자연사박물관(today)[1]
        서대문형무소역사관=parents.서대문형무소역사관(today)[1]
        서울시립미술관=parents.서울시립미술관(today)[1]
        선릉정릉=parents.선릉정릉(today)[1]
        종묘=parents.종묘(today)[1]
        창경궁=parents.창경궁(today)[1]
        창덕궁=parents.창덕궁(today)[1]
        태릉강릉조선왕릉전시관=parents.태릉강릉조선왕릉전시관(today)[1]
        트릭아이미술관=parents.트릭아이미술관(today)[1]
        헌릉인릉=parents.헌릉인릉(today)[1]

        df=df.append({'경복궁':경복궁,'국립중앙박물관':국립중앙박물관,'남산골한옥마을':남산골한옥마을,
                                    '덕수궁':덕수궁,'서대문자연사박물관':서대문자연사박물관,
                      '서대문형무소역사관':서대문형무소역사관,
                                    '서울시립미술관':서울시립미술관,
                      '선릉정릉':선릉정릉,'종묘':종묘,'창경궁':창경궁,
                                    '창덕궁':창덕궁,'태릉강릉조선왕릉전시관':태릉강릉조선왕릉전시관,
                      '트릭아이미술관':트릭아이미술관,
                                    '헌릉인릉':헌릉인릉}, ignore_index=True)
        df=df.transpose()
        df=df.sort_values(by=0, ascending=False)
        return df

In [ ]:
class weather_go:
    def make_list(today):
        df=child.make_dataframe(today)
        visual_list=list(df.index)
        return visual_list
    def make_wordcloud(today):
        visual_list=weather_go.make_list(today)
        wordcloud_list=[]
        for i in range(1,15):
            wordcloud_list.append(visual_list[0])
        for i in range(1,14):
            wordcloud_list.append(visual_list[1])
        for i in range(1,13):
            wordcloud_list.append(visual_list[2])
        for i in range(1,12):
            wordcloud_list.append(visual_list[3])
        for i in range(1,11):
            wordcloud_list.append(visual_list[4])
        for i in range(1,10):
            wordcloud_list.append(visual_list[5])
        for i in range(1,9):
            wordcloud_list.append(visual_list[6])
        for i in range(1,8):
            wordcloud_list.append(visual_list[7])
        for i in range(1,7):
            wordcloud_list.append(visual_list[8])
        for i in range(1,6):
            wordcloud_list.append(visual_list[9])
        for i in range(1,5):
            wordcloud_list.append(visual_list[10])
        for i in range(1,4):
            wordcloud_list.append(visual_list[11])
        for i in range(1,3):
            wordcloud_list.append(visual_list[12])
        for i in range(1,2):
            wordcloud_list.append(visual_list[13])

        
        text = " ".join(str(i) for i in wordckoud_list)
        wordcloud = WordCloud(font_path='NanumGothic', max_words=200,colormap='Set3', background_color="white").generate(text)
        plt.rcParams['font.family'] = 'NanumGothic' 
        plt.figure(figsize=(10,6))
        plt.figure(figsize=(15,10))
        plt.imshow(wordcloud, interpolation='Bilinear')
        plt.axis("off")
        plt.figure(1,figsize=(15, 15))
        plt.show() 

    def make_plot(today):
        df=child.make_dataframe(today)
        plt.rcParams['font.family'] = 'NanumGothic' 
        figure, ((ax1)) = plt.subplots(nrows=1, ncols=1)
        figure.set_size_inches(10,5)
        sns.barplot(data=df, x=df.index, y=df[0], ax=ax1)
        sns.set_style('darkgrid')
        ax1.set(ylabel="관광지추천지수", title="이번달 관광지추천")
        plt.show()

In [ ]:
weather_go.make_wordcloud(today)

[17:48:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


ValueError: ignored

In [ ]:
# 원래는 기온, 습도으로 해주어야 하나, 여기서는 평균 기온과 평균 습도로 열지수의 평균을 구해주도록 하겠다.
seoul['열지수']=0
i=0

for i in range(0,108):
    Tc=seoul.loc[i, '평균 기온(°C)']
    T=(Tc*1.8)+32
    R=seoul.loc[i, '평균 상대습도(%)']*1/100
    
    h1= -42.379
    h2=2.04901523*T
    h3=10.14333127*R
    h4=-0.22475541*T*R
    h5=-6.83783*(pow(10, -3))*((pow(T, 2)))
    h6=-5.481717*(pow(10, -2))*(pow(R, 2))
    h7=1.22874*(pow(10,-3))*T*R
    h8=8.5282*(pow(10, -4))*T*(pow(R, 2))
    h9=-1.99*(pow(10, -6))*(pow(T, 2))*(pow(R, 2))
    
    Heat_index=h1+h2+h3+h4+h5+h6+h7+h8+h9
    
    seoul['열지수'][i]=Heat_index
    i+=1

In [ ]:
today=seoul.iloc[103, :]
today

Unnamed: 0    1.030000e+02
date          2.019080e+05
year          2.019000e+03
month         8.000000e+00
평균 기온(°C)     2.719032e+01
                  ...     
트릭아이미술관       7.269000e+03
헌릉ㆍ인릉         3.928000e+03
선릉·정릉         2.962900e+04
월별합계          1.584299e+06
열지수           7.300000e+01
Name: 103, Length: 67, dtype: float64

In [ ]:
today=pd.DataFrame(today)

In [ ]:
today=today.transpose()

In [ ]:
today

,Unnamed: 0,date,year,month,평균 기온(°C),최저 기온(°C),최저 기온 시각(hhmi),최고 기온(°C),최고 기온 시각(hhmi),1시간 최다강수량 시각(hhmi),일강수량(mm),최대 순간풍속(m/s),최대 순간풍속 풍향(16방위),최대 순간풍속 시각(hhmi),최대 풍속(m/s),최대 풍속 풍향(16방위),최대 풍속 시각(hhmi),평균 풍속(m/s),풍정합(100m),최다 풍향(16방위),평균 이슬점온도(°C),최소 상대습도(%),평균 상대습도 시각(hhmi),평균 상대습도(%),평균 중기압(hPa),평균 현지기압(hPa),최고 해면 기압(hPa),최고 해면기압 시각(hhmi),최저 해면기압(hPa),최저 해면기압 시각(hhmi),평균 해면기압(hPa),가조시간(hr),합계 일조 시간(hr),1시간 최다 일사 시각(hhmi),1시간 최다 일사량(MJ/m2),합계 일사량(MJ/m2),평균 전운량(10분위),평균 중하층운량(10분위),평균 지면온도(°C),최저 초상온도(°C),평균 5cm 지중온도(°C),평균 10cm 지중온도(°C),평균 20cm 지중온도(°C),평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0cm 지중온도(°C),1.5cm 지중온도(°C),3.0cm 지중온도(°C),5.0cm 지중온도(°C),합계 소형증발량(mm),9-9강수(mm),경복궁,종묘,창경궁,창덕궁,남산골한옥마을,덕수궁,서울시립미술관 본관,국립중앙박물관,태릉 · 강릉 · 조선왕릉전시관,서대문자연사박물관,서대문형무소역사관,트릭아이미술관,헌릉ㆍ인릉,선릉·정릉,월별합계,열지수
103,103.0,201908.0,2019.0,8.0,27.190323,23.590323,696.580645,31.635484,1468.516129,4.822667,12.7,7.525806,216.129032,1440.774194,3.819355,239.354839,1544.548387,1.590323,1369.741935,180.645161,20.354839,44.645161,1358.548387,68.516129,24.235484,996.912903,1008.551613,1049.967742,1004.622581,1165.096774,1006.551613,13.6,7.976667,1283.333333,2.416,16.766897,5.632258,3.522581,28.932258,20.964516,27.951613,27.767742,27.219355,27.041935,26.841935,25.222581,23.790323,18.941935,15.316129,5.164516,9.664706,260658.0,29250.0,55413.0,70899.0,73788.0,149905.0,273904.0,478483.0,7079.0,37779.0,106315.0,7269.0,3928.0,29629.0,1584299.0,73.0
